In [57]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')
english_stopwords = stopwords.words('english')
combined_stopwords = list(set(turkish_stopwords + english_stopwords))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
import pandas as pd
train_classification = pd.read_csv('train-classification.csv')
train_classification = train_classification.rename(columns={'Unnamed: 0': 'username', 'label': 'label'})

train_classification["label"] = train_classification['label'].apply(str.lower)
username_category = train_classification.set_index('username').to_dict()['label']

# Load the new CSV file
annotated_users = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')

for index, row in annotated_users.iterrows():
    if index == 0:  # If it is the first row, skip
        continue
    username = row['username']
    if pd.isnull(row['influencerCategory']):
        continue
    influencer_category = row['influencerCategory'].strip().lower()  # Normalize (remove spaces and convert to lowercase)
    if username not in username_category:
        username_category[username] = influencer_category

# Apply the same process when adding the other file
annotated_users = pd.read_csv('annotated_users_CS412-0e2ff92120f3.csv')
for index, row in annotated_users.iterrows():
    if index == 0:  # If it is the first row, skip
        continue
    username = row['username']
    if pd.isnull(row['influencerCategory']):
        continue
    influencer_category = row['influencerCategory'].strip().lower()  # Normalize
    if username not in username_category:
        username_category[username] = influencer_category

# Additionally, normalize all existing categories
username_category = {k: v.strip().lower() for k, v in username_category.items()}


In [59]:
normalized_table = {
    'mom and children' : "Mom and Children",
    "food" : "Food",
    'travel' : "Travel",
    'fashion' : "Fashion",
    'gaming' : "Gaming",
    'health and lifestyle' : "Health and Lifestyle",
    'entertainment' : "Entertainment",
    'tech': "Tech",
    'sports' : "Sports",
    'art' : "Art"
}
normalized_table

{'mom and children': 'Mom and Children',
 'food': 'Food',
 'travel': 'Travel',
 'fashion': 'Fashion',
 'gaming': 'Gaming',
 'health and lifestyle': 'Health and Lifestyle',
 'entertainment': 'Entertainment',
 'tech': 'Tech',
 'sports': 'Sports',
 'art': 'Art'}

In [60]:
annotated_users = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')
annotated_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   username            150 non-null    object
 1   url                 150 non-null    object
 2   influencerCategory  147 non-null    object
 3   influencerMention   147 non-null    object
 4   accountType         147 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [61]:
username_category

{'taskirancemal': 'mom and children',
 'tam_kararinda': 'food',
 'spart4nn': 'food',
 'sosyalyiyiciler': 'food',
 'sonaydizdarahad': 'mom and children',
 'somersivrioglu': 'food',
 'sinankoc': 'travel',
 'simulasyonturk': 'gaming',
 'savas_karakas_sudaki_izler': 'travel',
 'sakinenurunannesi': 'mom and children',
 'ruyabuyuktetik': 'fashion',
 'raykakumru': 'health and lifestyle',
 'pintipanda': 'gaming',
 'pinarindepresyonu': 'health and lifestyle',
 'pinarhotic': 'health and lifestyle',
 'pinardonmez_': 'mom and children',
 'ozgeninoltasi': 'food',
 'nayaozgun': 'fashion',
 'nataliyarcan': 'health and lifestyle',
 'muthispsikoloji': 'health and lifestyle',
 'murattekecicom': 'tech',
 'mrsisbeceren': 'mom and children',
 'mosyosokola': 'food',
 'mirandaorlayn': 'entertainment',
 'mezarci_pubg': 'gaming',
 'mervedemireltaskiran': 'travel',
 'lifewithapineapple': 'travel',
 'lalsefkatli': 'travel',
 'kutupanne': 'mom and children',
 'kosifcihan': 'health and lifestyle',
 'kisikatescom':

In [62]:
len(username_category)

2890

In [63]:
train_classification.groupby('label').count()

,username
label,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [64]:
import json
import pandas as pd

train_data_path = "training-dataset.jsonl"

username_post_train = {}
username_profile_train = {}
username_biography_train = {}

username_post_test = {}
username_profile_test = {}
username_biography_test = {}

username_follower_count = {}  # Mapping of username -> follower_count

with open(train_data_path, "r", encoding="utf-8") as fh:
    for line in fh:
        sample = json.loads(line.strip())
        profile = sample["profile"]

        username = profile["username"]
        follower_count = profile.get("follower_count", 0)  # Follower count (default is 0 if missing)
        biography = profile.get("biography", "")           # Biography (default is "" if missing)

        # Separate into training or test sets based on the availability of category information
        if username in username_category:
            # Training
            username_post_train[username] = sample["posts"]
            username_profile_train[username] = profile
            username_biography_train[username] = biography
        else:
            # Test
            username_post_test[username] = sample["posts"]
            username_profile_test[username] = profile
            username_biography_test[username] = biography

        # Save follower count
        username_follower_count[username] = follower_count

# Convert profile information to a DataFrame
# `reset_index(drop=True)` creates a new integer index starting from zero.
train_profile_df = pd.DataFrame(username_profile_train).T.reset_index(drop=True)
test_profile_df  = pd.DataFrame(username_profile_test).T.reset_index(drop=True)

# Display the first 5 rows as an example
print(train_profile_df.head(5))


               username           id                     full_name  \
0            deparmedya   3170700063                   Depar Medya   
1            kafesfirin    266439571                   KAFES FIRIN   
2              vimerang   2367195567                      Vimerang   
3  yusufelibelediyesi08  43824173432  Yusufeli Belediye Başkanlığı   
4     mustafa_yalcinn38   9606564254               Mustafa Yalçın   

                                           biography   category_name  \
0           #mediaplanning #mediabuying #sosyalmedya  Local business   
1  📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...           Brand   
2       Dijital İletişim Yönetimi🎬info@vimerang.comq            None   
3  Yusufeli Belediye Başkanlığı Resmi İnstagram H...       Community   
4                            Talas Belediye Başkanı      Politician   

  post_count follower_count following_count is_business_account is_private  \
0       None           1167             192                True     

In [65]:
train_profile = pd.DataFrame(username_profile_train).T.reset_index(drop=True)
test_profile = pd.DataFrame(username_profile_test).T.reset_index(drop=True)

train_profile.head(5)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,...,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,...,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,vimerang,2367195567,Vimerang,Dijital İletişim Yönetimi🎬info@vimerang.comq,None,None,2321,454,True,False,...,Creators & Celebrities,None,VIDEO_CREATOR,False,False,https://instagram.fist19-1.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,yusufelibelediyesi08,43824173432,Yusufeli Belediye Başkanlığı,Yusufeli Belediye Başkanlığı Resmi İnstagram H...,Community,None,4635,83,True,False,...,None,None,None,False,False,https://instagram.fszf2-1.fna.fbcdn.net/v/t51....,False,False,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,mustafa_yalcinn38,9606564254,Mustafa Yalçın,Talas Belediye Başkanı,Politician,None,13647,29,True,False,...,None,None,POLITICIAN,False,False,https://instagram.fist1-4.fna.fbcdn.net/v/t51....,True,False,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [66]:
test_profile.head(5)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,konforyatak,8782109673,Konfor Yatak #KonforluUykular,"Konfor Yatak, birbirinden farklı özelliklere s...",Furniture,None,40334,2,True,False,...,None,None,FURNITURE,False,False,https://instagram.fyei6-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,ht_kulup,1950140344,HT KULÜP,Bloomberght - Habertürk Magazin,None,None,158877,69,True,False,...,None,None,TV_SHOW,False,False,https://instagram.fada2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,ajansspor,338611487,Ajansspor,"🏢 Saran Group \n🏟 Anında, tarafsız spor haberl...",News & media website,None,93193,286,True,False,...,None,None,NEWS_SITE,False,False,https://instagram.fadb2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [67]:
print(len(train_profile), len(test_profile))

2815 2600


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import re


def preprocessing_text(text):
    # If text is None, assign an empty string
    if text is None:
        text = ""
    
    # Convert text to lowercase
    text = text.casefold()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove non-alphanumeric characters except hashtags and mentions
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


corpus = []  # List to hold processed text data
training_usernames = []  # List to hold usernames

for username, post in username_post_train.items():
    training_usernames.append(username)

    # Clean the captions
    cleaned_captions = []
    for p in post:
        post_caption = p.get('caption', '')
        if post_caption is None:
            continue

        post_caption = preprocessing_text(post_caption)
        if post_caption != '':
            cleaned_captions.append(post_caption)

    # Clean and add the biography
    biography = username_biography_train.get(username, '')
    cleaned_biography = preprocessing_text(biography)

    # Combine captions and biography
    user_text = "\n".join(cleaned_captions + [cleaned_biography])
    corpus.append(user_text)


# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words=combined_stopwords, max_features=7500)

vectorizer.fit(corpus)

x_post_train = vectorizer.transform(corpus)
y_train = [username_category[username] for username in training_usernames]

feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a DataFrame
tfidf = pd.DataFrame(
    x_post_train.toarray(), 
    columns=feature_names, 
    index=username_post_train.keys()
)

# Remove a specific column ("bir") from the DataFrame
filtered_version = tfidf.drop("bir", axis=1)

# ---------------------------
# 3) Adding profile information and categories
# ---------------------------
# Profile information of all users
train_profile = pd.DataFrame(username_profile_train).T

# Extract category_name information
train_profile['category_name'] = train_profile['category_name'].str.split().str[0]
train_profile["category_name"] = train_profile["category_name"].fillna("unknown")  # Fill missing categories

# One-hot encode the categories
encoder = OneHotEncoder()
category_features = encoder.fit_transform(train_profile["category_name"].values.reshape(-1, 1))

# Convert one-hot encoded results to a DataFrame
category_features_df = pd.DataFrame(
    category_features.toarray(),
    columns=encoder.get_feature_names_out(["category_name"]),
    index=train_profile.index
)

# Combine `filtered_version` with category features
combined_features = pd.concat([filtered_version, category_features_df], axis=1)

# ---------------------------
# 4) TF-IDF vectorization for the test set (Caption + Biography)
# ---------------------------
test_corpus = []  # List to hold processed text data for test set
test_usernames = []  # List to hold test usernames

for username, post in username_post_test.items():
    test_usernames.append(username)

    # Clean the captions
    cleaned_captions = []
    for p in post:
        post_caption = p.get('caption', '')
        if post_caption is None:
            continue

        post_caption = preprocessing_text(post_caption)
        if post_caption != '':
            cleaned_captions.append(post_caption)

    # Clean and add the biography
    biography = username_biography_test.get(username, '')
    if biography is None:
        biography = ''
        
    cleaned_biography = preprocessing_text(biography)

    # Combine captions and biography
    user_text = "\n".join(cleaned_captions + [cleaned_biography])
    test_corpus.append(user_text)

x_post_test = vectorizer.transform(test_corpus)

# Convert the test TF-IDF matrix to a DataFrame
test_tfidf = pd.DataFrame(
    x_post_test.toarray(), 
    columns=feature_names, 
    index=username_post_test.keys()
)


In [69]:
assert y_train.count("NA") == 0

In [70]:
if "bir" in filtered_version.columns:
    print("True")
else:
    print("False")

False


In [71]:
filtered_version.shape

(2815, 7499)

In [72]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords and combine them
nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')
english_stopwords = stopwords.words('english')
combined_stopwords = list(set(turkish_stopwords + english_stopwords))

# Text preprocessing function
def preprocessing_text(text):
    if text is None:
        text = ""
    text = text.casefold()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create the caption corpus
def build_caption_corpus(username_post):
    caption_corpus = []
    usernames = []

    for username, posts in username_post.items():
        usernames.append(username)
        cleaned_captions = [
            preprocessing_text(post.get('caption', ''))
            for post in posts if post.get('caption', '')
        ]
        user_text = "\n".join(cleaned_captions)
        caption_corpus.append(user_text)
    return caption_corpus, usernames

# Create the biography corpus
def build_biography_corpus(username_biography):
    biography_corpus = []
    usernames = []

    for username, biography in username_biography.items():
        usernames.append(username)
        cleaned_biography = preprocessing_text(biography)
        biography_corpus.append(cleaned_biography)
    return biography_corpus, usernames

# Create separate Caption and Biography corpora
caption_corpus, caption_usernames = build_caption_corpus(username_post_train)
biography_corpus, biography_usernames = build_biography_corpus(username_biography_train)

# Define and train separate TF-IDF vectorizers
caption_vectorizer = TfidfVectorizer(stop_words=combined_stopwords, max_features=7500)
biography_vectorizer = TfidfVectorizer(stop_words=combined_stopwords, max_features=2500)

# Fit and transform the data
x_caption_train = caption_vectorizer.fit_transform(caption_corpus)
x_biography_train = biography_vectorizer.fit_transform(biography_corpus)

# Retrieve feature names
caption_feature_names = caption_vectorizer.get_feature_names_out()
biography_feature_names = biography_vectorizer.get_feature_names_out()

# Convert the TF-IDF matrices to DataFrames
caption_tfidf = pd.DataFrame(
    x_caption_train.toarray(),
    columns=caption_feature_names,
    index=caption_usernames
)

biography_tfidf = pd.DataFrame(
    x_biography_train.toarray(),
    columns=biography_feature_names,
    index=biography_usernames
)

# Combine TF-IDF matrices
combined_tfidf = pd.concat([caption_tfidf, biography_tfidf], axis=1).fillna(0)

# Combine profile information and categories
train_profile = pd.DataFrame(username_profile_train).T
train_profile['category_name'] = train_profile['category_name'].str.split().str[0]
train_profile['category_name'] = train_profile['category_name'].fillna('unknown')

# One-hot encode the categories
encoder = OneHotEncoder()
category_features = encoder.fit_transform(train_profile['category_name'].values.reshape(-1, 1))
category_features_df = pd.DataFrame(
    category_features.toarray(),
    columns=encoder.get_feature_names_out(['category_name']),
    index=train_profile.index
)

# Combine TF-IDF and category features
combined_features = pd.concat([combined_tfidf, category_features_df], axis=1).fillna(0)

# Check and clean class distribution
valid_usernames = [username for username in username_category.keys() if username in combined_features.index]
filtered_combined_features = combined_features.loc[valid_usernames]
filtered_y_train = [username_category[username] for username in valid_usernames]

class_counts = Counter(filtered_y_train)
filtered_data = [(x, y) for x, y in zip(filtered_combined_features.index, filtered_y_train) if class_counts[y] > 1]
filtered_combined_features = filtered_combined_features.loc[[x for x, y in filtered_data]]
filtered_y_train = [y for x, y in filtered_data]

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    filtered_combined_features,
    filtered_y_train,
    test_size=0.2,
    stratify=filtered_y_train
)

# Train and evaluate the Naive Bayes model
model = MultinomialNB()
model.fit(x_train, y_train)

# Evaluate performance
y_train_pred = model.predict(x_train)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Train Classification Report:\n", classification_report(y_train, y_train_pred, zero_division=0))

y_val_pred = model.predict(x_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred, zero_division=0))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Train Accuracy: 0.7800977343402932
Train Classification Report:
                       precision    recall  f1-score   support

                 art       0.97      0.38      0.55       157
       entertainment       0.66      0.75      0.70       265
             fashion       0.86      0.83      0.84       244
                food       0.88      0.96      0.92       414
              gaming       0.00      0.00      0.00        14
health and lifestyle       0.62      0.91      0.74       413
    mom and children       0.97      0.23      0.37       121
              sports       0.98      0.50      0.66       100
                tech       0.83      0.91      0.87       287
              travel       0.89      0.76      0.82       236

            accuracy                           0.78      2251
           macro avg       0.77      0.62      0.65      2251
        weighted avg       0.81      0.78      0.76      2251

Validation Accuracy: 0.6252220248667851
Validation Classificatio

In [73]:
test_data_path = "test-classification-round2.dat"

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

['schirogomme', 'takviyegiller', 'egeasalmetal_', 'expomedeurasia', 'izmirhim']


In [74]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = training_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)

# remove screenname
#test_unames.remove("screenname")

In [75]:
len(x_test)

1000

In [76]:
len(test_unames)

1000

In [77]:
import numpy as np

# List of all features used in training
training_feature_names = combined_features.columns

# Convert test data to a DataFrame
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

# Add missing columns and fill with 0
for col in training_feature_names:
    if col not in df_test.columns:
        df_test[col] = 0

# Arrange columns in the same order
df_test = df_test[training_feature_names]

# Make predictions
test_pred = model.predict(df_test)

# Generate output
output = dict()
for index, uname in enumerate(test_unames):  # Replace test_unames with test_usernames
    output[uname] = test_pred[index]

# Check the output
print(output)


C:\Users\ganga\AppData\Local\Temp\ipykernel_14796\1641931476.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = 0
C:\Users\ganga\AppData\Local\Temp\ipykernel_14796\1641931476.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = 0
C:\Users\ganga\AppData\Local\Temp\ipykernel_14796\1641931476.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

{'schirogomme': 'tech', 'takviyegiller': 'health and lifestyle', 'egeasalmetal_': 'tech', 'expomedeurasia': 'tech', 'izmirhim': 'health and lifestyle', 'hotelveneraistanbul': 'travel', 'thepearlofgreece': 'fashion', 'ferrerorocherturkiye': 'food', 'meyhanedeyiz.biz': 'food', 'ozatashipyard': 'tech', 'cartmancase': 'tech', 'thermexturkiye': 'tech', 'kedi7_com': 'food', 'frostoys': 'mom and children', 'kayagrubu': 'tech', 'tavukdunyasikariyer': 'health and lifestyle', 'sukrugencbaskan': 'health and lifestyle', 'oguzhankapilar': 'entertainment', 'yrefahesenyurt': 'health and lifestyle', 'dedemanpalandokenskilodge': 'travel', 'fideyoapp': 'entertainment', 'fiatkastamonugozde': 'tech', 'nakisgidaofficial': 'health and lifestyle', 'instakocaali': 'travel', 'refahist': 'health and lifestyle', 'umutayildiz': 'health and lifestyle', 'kadikoybelediye': 'health and lifestyle', 'fanatikcomtr': 'sports', 'ecift': 'entertainment', 'kumsalistanbul': 'food', 'wenotrat': 'entertainment', 'alaralokum': 

In [78]:
len(df_test)

1000

In [79]:
# Filter columns that start with 'category_name'
category_columns = df_test.columns[df_test.columns.str.startswith('category_name')]

# Find columns with values different from 0
non_zero_columns = category_columns[df_test[category_columns].ne(0).any()].tolist()

# Print the result
print(non_zero_columns)


[]


In [80]:
df_test.columns
category_columns = []
for col in df_test.columns:
    if col.startswith('category_name'):
        category_columns.append(col)

category_columns

['category_name_Actor',
 'category_name_Advertising',
 'category_name_Advertising/Marketing',
 'category_name_Aerospace',
 'category_name_Agricultural',
 'category_name_Agriculture',
 'category_name_Animal',
 'category_name_Apartment',
 'category_name_App',
 'category_name_Aquarium',
 'category_name_Architectural',
 'category_name_Art',
 'category_name_Artist',
 'category_name_Arts',
 'category_name_Athlete',
 'category_name_Author',
 'category_name_Automation',
 'category_name_Automotive',
 'category_name_Aviation',
 'category_name_Baby',
 'category_name_Bags/Luggage',
 'category_name_Bakery',
 'category_name_Bar',
 'category_name_Beach',
 'category_name_Beauty',
 'category_name_Beauty,',
 'category_name_Bicycle',
 'category_name_Biotechnology',
 'category_name_Blinds',
 'category_name_Blogger',
 'category_name_Boat',
 'category_name_Book',
 'category_name_Bookstore',
 'category_name_Brand',
 'category_name_Bridal',
 'category_name_Broadcasting',
 'category_name_Building',
 'category_

In [81]:
normalized_table

{'mom and children': 'Mom and Children',
 'food': 'Food',
 'travel': 'Travel',
 'fashion': 'Fashion',
 'gaming': 'Gaming',
 'health and lifestyle': 'Health and Lifestyle',
 'entertainment': 'Entertainment',
 'tech': 'Tech',
 'sports': 'Sports',
 'art': 'Art'}

In [82]:
test_pred = model.predict(df_test)

output_normalized = dict()

for index, uname in enumerate(test_unames):
    predicted_label = test_pred[index].strip().lower()  # Convert the predicted category to lowercase
    normalized_label = normalized_table.get(predicted_label, predicted_label.title())  # Normalize or default to title case
    output_normalized[uname] = normalized_label


In [83]:
len(test_pred)

1000

In [84]:
with open("prediction-classification-round2.json", "w") as of:
  json.dump(output_normalized, of, indent=4)

In [85]:
# check for data inbalance
train_classification.groupby('label').count()

,username
label,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [86]:
test_usernames

['beyazyakaliyiz',
 'totalenergies_istasyonlari',
 'konforyatak',
 'ht_kulup',
 'ajansspor',
 '4bros.tr',
 'drtubagunebak',
 'nihatcan11',
 'mustafaaakcay',
 'mtmofamily',
 'bonitoperde',
 'balkanlardangelenlezzet',
 'pedagoganne__gulozturk',
 'iktm34',
 'yudumyag',
 'bulentozdemir.edebiyat',
 'imtolstoyevski',
 'yorkkadikoy',
 'akkaalinda',
 'mutfaktayusufvar',
 'antmodern',
 'immergasturkiye',
 'vanillaantalya',
 'ermanyasar',
 'tuna.food',
 'rustik.rus.restorani',
 'ilhansen',
 'institutfrancaisdeturquie',
 'beachandbeyondswimwear',
 'monsternotebook',
 'bsynctr',
 'kbbzone',
 'dbbanyo',
 'istanbulalpplastik',
 'enka_insaat_official',
 'ozyufka.com_tr',
 'nationalturk',
 'sametkaankuyucu',
 'tutbelediyesi',
 'odurla',
 'balagency',
 'sokebelediyesi',
 'dedemanpalandokenskilodge',
 'binefismutfak',
 'tatli.sanati',
 'evimin.yemekleri',
 'ceyo_turkiye',
 'berlinplatzpub.kadikoy',
 'rotaradar',
 'luppotr',
 'bogaziciyoneticilervakfi',
 'sestri.lifestyle',
 'sarduvantiyatrosu',
 'sevinc

In [87]:
# check for test prediction distribution
pd.Series(test_pred).value_counts()

health and lifestyle    351
food                    174
entertainment           146
tech                    139
travel                   92
fashion                  85
sports                    6
art                       5
mom and children          2
Name: count, dtype: int64

## Regression Part

In [88]:
from pprint import pprint

test_regression_datapath = "test-regression-round2.jsonl"


test_regression_data = []
with open(test_regression_datapath, "rt") as fh:
  for line in fh:
    test_regression_data.append(json.loads(line.strip()))

pprint(test_regression_data[995])

{'caption': 'Nergiz 34 yaşında Özbekistan\n'
            "6 yıldır Türkiye'de.\n"
            '3 yıl villa ,temizlik yemek deneyimleri var. \n'
            "Ümraniye'de oturuyor.\n"
            'Ayakta anne yada temizlik işleri istiyor.\n'
            'Oturum izni ve\xa0referansı\xa0var.\n'
            '\n'
            'Adayımızla ilgili bizimle iletişime geçebilirsiniz 😊🍀 \n'
            '\n'
            'Erenköy Danışmanlık \n'
            '0216 363 13 73\n'
            'www.erenkoydanismanlik.com \n'
            '\n'
            '#bakıcı #bakici #cocukbakicisi #özbekbakıcı #türkmenbakıcı '
            '#filipinlibakıcı #afrikalıbakıcı #ingilizcebilenbakıcı '
            '#danışmanlık #yaşlıbakıcısı #hastabakıcı #avrupabakıcı '
            '#avrupahastabakici',
 'comments_count': 0,
 'id': '18115268470322981',
 'media_type': 'IMAGE',
 'media_url': 'https://scontent-sof1-1.cdninstagram.com/v/t51.29350-15/395473950_1063251994801508_7273187389115076453_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=

In [89]:
from collections import defaultdict

def predict_like_count(username, current_post=None):
    def get_avg_like_count_by_media_type(posts: list):
        media_type_likes = defaultdict(list)
        for post in posts:
            # Skip the current post if provided
            if current_post is not None and post["id"] == current_post["id"]:
                continue
            
            media_type = post.get("media_type", "UNKNOWN")
            like_count = post.get("like_count", 0)
            if like_count is None:
                like_count = 0
            media_type_likes[media_type].append(like_count)
        
        avg_likes_by_media_type = {}
        for media_type, likes in media_type_likes.items():
            # Calculate average likes for each media type
            if likes:
                avg_likes_by_media_type[media_type] = sum(likes) / len(likes)
            else:
                avg_likes_by_media_type[media_type] = 0.0
        
        return avg_likes_by_media_type

    if username in username_post_train:
        avg_likes = get_avg_like_count_by_media_type(username_post_train[username])
    elif username in username_post_test:
        avg_likes = get_avg_like_count_by_media_type(username_post_test[username])
    else:
        print(f"No data available for {username}")
        return 0  # Returns 0 instead of -1

    if current_post and current_post.get("media_type") in avg_likes:
        # If the current post exists and its media_type is known, use its specific average
        prediction = avg_likes[current_post["media_type"]]
    else:
        # If the current post is not provided or media_type is unknown, return the general average
        prediction = sum(avg_likes.values()) / len(avg_likes) if avg_likes else 0.0

    # Ensure the prediction is not less than 0
    return max(0, prediction)


In [90]:
def log_mse_like_counts_log10(y_true, y_pred):
    """
    Calculate the Log Mean Squared Error (Log MSE) for like counts (log10(like_count + 1)).

    Parameters:
    - y_true: array-like, actual like counts
    - y_pred: array-like, predicted like counts

    Returns:
    - log_mse: float, Log Mean Squared Error
    """
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Log transformation: log10(like_count + 1)
    log_y_true = np.log10(y_true + 1)
    log_y_pred = np.log10(y_pred + 1)

    # Compute squared errors
    squared_errors = (log_y_true - log_y_pred) ** 2

    # Return the mean of squared errors
    return np.mean(squared_errors)


In [91]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username_post_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts_log10(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 0.1986404001356902


In [92]:
import json

path = "test-regression-round2.jsonl"
output_path = "prediction-regression-round2.json"

# We will store the results in a dictionary
output_dict = {}

with open(path, "rt") as fh:
    for line in fh:
        sample = json.loads(line)

        # Make a prediction
        pred_val = predict_like_count(sample["username"])
        sample["like_count"] = int(pred_val)

        # Add the ID as the key and the predicted value as the value
        output_dict[sample["id"]] = sample["like_count"]

# Write the JSON in the specified format
with open(output_path, "wt") as of:
    json.dump(output_dict, of, indent=4)

In [93]:
from pprint import pprint
pprint(output_dict)

{'17842125969098625': 68,
 '17842133421053680': 19,
 '17842262878347614': 41,
 '17842536582014367': 37,
 '17842616685097835': 69,
 '17842698372057811': 29,
 '17842843491082799': 94,
 '17842964729217076': 50,
 '17842971761056319': 146,
 '17843011057181888': 78,
 '17843911590088401': 68,
 '17843914980035156': 106,
 '17844053652063317': 489,
 '17844113249230178': 10,
 '17844150115795324': 190,
 '17844197268085344': 237,
 '17844304338099142': 10862,
 '17844425537880934': 378,
 '17844437155305173': 27,
 '17844438131457959': 6,
 '17844573736118484': 11,
 '17845148913054711': 36,
 '17845290365889769': 39,
 '17845321360490579': 101,
 '17845574133093613': 55,
 '17845625072592235': 41,
 '17845713315070947': 154,
 '17846057727068087': 69,
 '17846418540053860': 44,
 '17846468333175606': 56582,
 '17846613890183781': 52,
 '17846617312891802': 152,
 '17846810796093875': 24,
 '17846946897053786': 78,
 '17847341364055961': 50,
 '17847481089011752': 3,
 '17847600461892145': 26,
 '17847671346081559': 45,

In [94]:
my_test_df = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')
my_test_df

,username,url,influencerCategory,influencerMention,accountType
0,santrasosyal,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Health and Lifestyle,No,Company
1,sortiesortie,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Entertainment,No,Company
2,bodrumlupastalar,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Food,No,Company
3,ihealthsaglik,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Health and Lifestyle,No,Company
4,bisikletyolculugu,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Sports,Yes,Influencer
...,...,...,...,...,...
145,fringeistanbul,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Entertainment,No,Company
146,46oyunculari,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Art,No,Company
147,twitchdenkesitler,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Gaming,No,Influencer
148,stmdefence,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Tech,No,Company


In [95]:
len(username_category)

2890

In [96]:
len(output_dict)

3000